## Set up matching DF's

In [51]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)
import os

In [52]:
directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/Toto"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_toto = max(files, key=os.path.getmtime)

directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/unibet"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_unibet = max(files, key=os.path.getmtime)

In [53]:
latest_file_unibet

'/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/unibet/unibetAllSport2025-02-02 11:34:56.895611.csv'

In [54]:
toto_raw = pd.read_csv(f"{latest_file_toto}", index_col=0).drop_duplicates()
kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
# Apply the condition and update outcome_label
toto_raw_football = toto_raw[toto_raw['sport'] == 'Voetbal']
kambi_raw_football = kambi_raw[kambi_raw['sport'] == 'FOOTBALL']
# Adjust odds and line
kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
kambi_raw_football['odds'] = kambi_raw_football['odds'] / 1000

# List of women's competitions
women_competitions_toto = [
    'Portugal Campeonato Nacional, Vrouwen', 'Mexico League MX Vrouwen', 'Australië W-League', 'Italië Coppa Italia Vrouwen',
    'Scotland Women\'s Premier League', 'Nederland Eredivisie Vrouwen', 'England FA Cup Women', 'Engeland FA Super League Vrouwen',
    'Spain Primera División Vrouwen'
]

# Create 'sex' column based on the competition
toto_raw_football['sex'] = toto_raw_football['competition'].apply(
    lambda x: 'W' if x in women_competitions_toto else 'M'
)

# List of women's football competitions
women_competitions_kambi = [
    'A-League (D)', 'Premier League Dames', 'Campeonato Nacional Feminino', 'Liga MX Femenil (D)', 
    'Frauen-Bundesliga', 'Super League (D)', 'Primera División (D)', 'Coppa Italia (D)', 'Liga MX Femenil'

]

# Create 'sex' column
kambi_raw_football['sex'] = kambi_raw_football['group_name'].apply(
    lambda x: 'W' if x in women_competitions_kambi else 'M'
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/3696476550.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/3696476550.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/3696476550.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [103]:
kambi_raw[kambi_raw['event_name'].str.contains('Club Brugge')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name
67157,2489553704,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1022540293,3626720229,1X,1X,1420.0,NaN,NaN,OT_ONE_OR_CROSS,2025-02-01T18:28:18Z,21/50,-240.0,OPEN,ENABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie
67158,2489553704,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1022540293,3626720233,12,12,1550.0,NaN,NaN,OT_ONE_OR_TWO,2025-02-01T18:28:18Z,11/20,-182.0,OPEN,ENABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie
67159,2489553704,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1022540293,3626720241,X2,X2,1300.0,NaN,NaN,OT_CROSS_OR_TWO,2025-02-01T18:28:18Z,3/10,-335.0,OPEN,ENABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie
67160,2489553705,1002558602,Asian Totaal - 1e Helft,Asian Total - 1st Half,GOALS,NaN,21,Asian Over/Under,Asian Over/Under,1022540293,3626720230,Meer dan,Over,1410.0,750.0,NaN,OT_OVER,2025-02-01T18:28:18Z,2/5,-245.0,OPEN,DISABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie
67161,2489553705,1002558602,Asian Totaal - 1e Helft,Asian Total - 1st Half,GOALS,NaN,21,Asian Over/Under,Asian Over/Under,1022540293,3626720236,Minder dan,Under,2850.0,750.0,NaN,OT_UNDER,2025-02-01T18:28:18Z,37/20,185.0,OPEN,DISABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68378,2489950133,1001243173,Totaal Aantal Doelpunten - 2e Helft,Total Goals - 2nd Half,GOALS,NaN,6,Over/Onder,Over/Under,1020999872,3628108987,Meer dan,Over,3800.0,2500.0,NaN,OT_OVER,2025-02-02T01:15:39Z,14/5,280.0,OPEN,ENABLED,NaN,NaN,Royal Antwerp FC vs Club Brugge,FOOTBALL,Jupiler Pro League
68379,2489950134,1001243173,Totaal Aantal Doelpunten - 2e Helft,Total Goals - 2nd Half,GOALS,NaN,6,Over/Onder,Over/Under,1020999872,3628108988,Minder dan,Under,1040.0,3500.0,NaN,OT_UNDER,2025-02-02T01:15:38Z,1/25,-2500.0,OPEN,ENABLED,NaN,NaN,Royal Antwerp FC vs Club Brugge,FOOTBALL,Jupiler Pro League
68380,2489950134,1001243173,Totaal Aantal Doelpunten - 2e Helft,Total Goals - 2nd Half,GOALS,NaN,6,Over/Onder,Over/Under,1020999872,3628108989,Meer dan,Over,8000.0,3500.0,NaN,OT_OVER,2025-02-02T01:15:38Z,7/1,700.0,OPEN,ENABLED,NaN,NaN,Royal Antwerp FC vs Club Brugge,FOOTBALL,Jupiler Pro League
68381,2489950135,1005153925,Scoort ten minste 2 doelpunten,To score at least 2 goals,GOALS,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1020999872,3628108990,Ja,Yes,31000.0,2000.0,"Jashari, Ardon",OT_YES,2025-02-02T07:45:41Z,30/1,3000.0,OPEN,DISABLED,NaN,NaN,Royal Antwerp FC vs Club Brugge,FOOTBALL,Jupiler Pro League


Below is a translation of the abbreviations for the column outcome types into their likely meanings in the context of betting odds:

- `--`: Undefined or No Outcome (e.g., no odds available).
- `AG`: Anytime Goal Scorer (player to score at any time in the match).
- `CS`: Correct Score (predict the exact final score of the event).
- `DC`: Double Chance (cover two outcomes, e.g., Home Win or Draw).
- `DN`: Draw No Bet (stake refunded if the match ends in a draw).
- `FS`: First Scorer (player to score the first goal/point in the event).
- `H1`: First Half Result (outcome at the end of the first half).
- `H2`: Second Half Result (outcome in the second half only).
- `HF`: Half-Time/Full-Time (predict the result at both half-time and full-time).
- `HH`: Head-to-Head (comparison between two participants, e.g., players or teams).
- `HL`: Handicap Line (spread betting; adjust the line to level the playing field).
- `LS`: Last Scorer (player to score the last goal/point in the event).
- `MH`: Match Handicap (spread betting for the entire match).
- `MR`: Match Result (predict the overall winner or draw for the match).
- `OE`: Odd/Even (predict whether the total points/goals scored will be odd or even).
- `WH`: Winning Half (which half will have the higher score).
- `WM`: Winning Margin (predict the margin by which a team/player will win).

In [56]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_filtered_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_football based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_football based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

# Replace 'Sport Club' with 'SC' in the 'event_name' column
kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/1898072072.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)


## Winnaar: Draw no Bet

In [57]:
from rapidfuzz import process, fuzz
import pandas as pd

# Filter kambi_filtered_football based on criterion_label
filtered_kambi_winnaar = kambi_filtered_football[
    kambi_filtered_football['criterion_label'].str.contains('Draw No Bet')
]

# Filter kambi_filtered_football based on criterion_label
filtered_toto_winnaar = toto_filtered_football[
    toto_filtered_football['Market Name'].str.contains('Draw No Bet')
]

# Preprocess strings: Remove extra whitespace and lowercase
def preprocess(text):
    return ' '.join(text.lower().split())

In [58]:
# import unicodedata

# # Replace '-' with a space and remove accents in the 'names' column
# filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
#     lambda x: ''.join(
#         char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
#         if not unicodedata.combining(char)
#     )
# )
# # Replace '-' with a space and remove accents in the 'names' column
# filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
#     lambda x: ''.join(
#         char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
#         if not unicodedata.combining(char)
#     )
# )

# # Create 'Team1' and 'Team2'
# filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
# filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

# kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score, _ = result
#     return match

# # Apply matching function to toto_filtered_football
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

In [104]:
import unicodedata
from fuzzywuzzy import process, fuzz

# Replace '-' with a space and remove accents in the 'names' column
filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Create 'Team1' and 'Team2'
filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match based on Team1 and Team2 fuzzy matching
# def find_best_match(event_name):
#     # Extract Team1 and Team2 from the event_name
#     parts = event_name.split(' vs ')
#     if len(parts) == 2:
#         team1 = parts[0].strip()
#         team2 = parts[1].strip()

#         # Perform fuzzy matching for Team1 and Team2
#         def match_teams(team, kambi_team):
#             return fuzz.token_set_ratio(team, kambi_team) >= 80

#         # Check each event_name in kambi_events
#         for kambi_event in kambi_events:
#             kambi_parts = kambi_event.split(' vs ')
#             if len(kambi_parts) == 2:
#                 kambi_team1 = kambi_parts[0].strip()
#                 kambi_team2 = kambi_parts[1].strip()

#                 # Check if both teams match
#                 if match_teams(team1, kambi_team1) and match_teams(team2, kambi_team2):
#                     return kambi_event  # Return the matched event_name

#     return None  # Return None if no match found

from fuzzywuzzy import fuzz

# Function to find the best match based on Team1 and Team2 fuzzy matching
def find_best_match(event_name):
    # Extract Team1 and Team2 from the event_name
    parts = event_name.split(' vs ')
    if len(parts) == 2:
        team1 = parts[0].strip().replace('Athletic', '').replace('Atletico', '')
        team2 = parts[1].strip().replace('Athletic', '').replace('Atletico', '')

        # Perform fuzzy matching for Team1 and Team2 with all kambi events
        def match_teams(team, kambi_team):
            return fuzz.token_set_ratio(team, kambi_team)

        # Perform fuzzy matching for both Team1 and Team2
        kambi_matches = filtered_kambi_winnaar[
            (filtered_kambi_winnaar['Team1'].apply(lambda x: match_teams(team1, x) >= 65)) &
            (filtered_kambi_winnaar['Team2'].apply(lambda x: match_teams(team2, x) >= 65))
        ]
        
        # If there is a match, return the matched event_name and fuzzy match score
        if not kambi_matches.empty:
            # Get the best match score
            best_score_team1 = kambi_matches['Team1'].apply(lambda x: match_teams(team1, x)).max()
            best_score_team2 = kambi_matches['Team2'].apply(lambda x: match_teams(team2, x)).max()

            # Calculate the average score (or you can choose to return another metric)
            average_score = (best_score_team1 + best_score_team2) / 2

            # Get the matched event name
            matched_event_name = kambi_matches['event_name'].iloc[0]

            return matched_event_name, average_score  # Return the matched event and score

    return None, None  # Return None if no match found

# Apply matching function to toto_filtered_football
filtered_toto_winnaar[['matched_event', 'fuzzy_score']] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: pd.Series(find_best_match(x))
)

# # Apply matching function to filtered_toto_winnaar
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

# Check the results
filtered_toto_winnaar[['Event Name', 'matched_event']].head()

,Event Name,matched_event
event_id,,
6360273,1. FC Kaiserslautern vs SC Preussen Munster,1. FC Kaiserslautern vs Preußen Munster
6360273,1. FC Kaiserslautern vs SC Preussen Munster,1. FC Kaiserslautern vs Preußen Munster
6360268,Hamburger SV vs Hannover 96,Hamburger SV vs Hannover 96
6360268,Hamburger SV vs Hannover 96,Hamburger SV vs Hannover 96
6360269,SC Paderborn vs SpVgg Greuther Furth,SC Paderborn 07 vs Greuther Furth


In [100]:
filtered_toto_winnaar[filtered_toto_winnaar['Event Name'].str.contains('Brugge')]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,sex,Team1,Team2,matched_event,fuzzy_score,standardized_label
event_id,,,,,,,,,,,,,,,,,,,
6361163,Royal Antwerp FC vs Club Brugge,Draw No Bet,Royal Antwerp FC,3.30,23,10,DN,1,Voetbal,België Jupiler Pro League,Royal Antwerp FC vs Club Brugge,Royal Antwerp FC,Club Brugge,M,Royal Antwerp FC,Club Brugge,Royal Antwerp FC vs Club Brugge | Royal Antwer...,100.0,Draw No Bet
6361163,Royal Antwerp FC vs Club Brugge,Draw No Bet,Club Brugge,1.34,17,50,DN,2,Voetbal,België Jupiler Pro League,Royal Antwerp FC vs Club Brugge,Royal Antwerp FC,Club Brugge,M,Royal Antwerp FC,Club Brugge,Royal Antwerp FC vs Club Brugge | Royal Antwer...,100.0,Draw No Bet
6416613,KRC Genk vs Cercle Brugge,Draw No Bet,KRC Genk,1.24,6,25,DN,1,Voetbal,België Jupiler Pro League,KRC Genk vs Cercle Brugge,KRC Genk,Cercle Brugge,M,KRC Genk,Cercle Brugge,KRC Genk vs Club Brugge | KRC Genk vs Club Bru...,87.5,Draw No Bet
6416613,KRC Genk vs Cercle Brugge,Draw No Bet,Cercle Brugge,3.60,13,5,DN,2,Voetbal,België Jupiler Pro League,KRC Genk vs Cercle Brugge,KRC Genk,Cercle Brugge,M,KRC Genk,Cercle Brugge,KRC Genk vs Club Brugge | KRC Genk vs Club Bru...,87.5,Draw No Bet
6737084,Club Brugge vs Atalanta,Draw No Bet,Atalanta,1.42,21,50,DN,2,Voetbal,UEFA Champions League,Club Brugge vs Atalanta,Club Brugge,Atalanta,M,Club Brugge,Atalanta,None,NaN,Draw No Bet
6737084,Club Brugge vs Atalanta,Draw No Bet,Club Brugge,2.65,33,20,DN,1,Voetbal,UEFA Champions League,Club Brugge vs Atalanta,Club Brugge,Atalanta,M,Club Brugge,Atalanta,None,NaN,Draw No Bet


In [60]:
filtered_kambi_winnaar

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,sex,Team1,Team2
692,2488927083,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021130332,3624456638,1,1,1.09,NaN,Universitatea Craiova,1,2025-02-02T10:18:25Z,1/12,-1115.0,OPEN,DISABLED,NaN,NaN,Universitatea Craiova vs Politehnica Iasi,FOOTBALL,Liga I,M,Universitatea Craiova,Politehnica Iasi
693,2488927083,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021130332,3624456647,2,2,7.00,NaN,Politehnica Iași,2,2025-02-02T10:18:25Z,6/1,600.0,OPEN,DISABLED,NaN,NaN,Universitatea Craiova vs Politehnica Iasi,FOOTBALL,Liga I,M,Universitatea Craiova,Politehnica Iasi
709,2488927095,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021130332,3624456662,1,1,1.15,NaN,Universitatea Craiova,1,2025-02-02T10:18:26Z,1/7,-670.0,OPEN,DISABLED,NaN,NaN,Universitatea Craiova vs Politehnica Iasi,FOOTBALL,Liga I,M,Universitatea Craiova,Politehnica Iasi
710,2488927095,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021130332,3624456675,2,2,4.40,NaN,Politehnica Iași,2,2025-02-02T10:18:26Z,17/5,340.0,OPEN,DISABLED,NaN,NaN,Universitatea Craiova vs Politehnica Iasi,FOOTBALL,Liga I,M,Universitatea Craiova,Politehnica Iasi
816,2488942707,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021130335,3624556478,1,1,1.68,NaN,FCSB,1,2025-02-01T16:54:23Z,4/6,-148.0,OPEN,DISABLED,NaN,NaN,FCSB vs CFR Cluj,FOOTBALL,Liga I,M,FCSB,CFR Cluj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90047,2488721103,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021103884,3623904395,2,2,13.00,NaN,Ross County,2,2025-01-30T18:44:58Z,12/1,1200.0,OPEN,DISABLED,NaN,NaN,Rangers FC vs Ross County,FOOTBALL,Schotse Premiership,M,Rangers FC,Ross County
90224,2488757056,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021103898,3623883630,1,1,9.50,NaN,Motherwell,1,2025-02-01T20:57:17Z,17/2,850.0,OPEN,DISABLED,NaN,NaN,Motherwell vs Celtic,FOOTBALL,Schotse Premiership,M,Motherwell,Celtic
90225,2488757056,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021103898,3623883636,2,2,1.05,NaN,Celtic,2,2025-02-01T20:57:17Z,1/20,-2000.0,OPEN,DISABLED,NaN,NaN,Motherwell vs Celtic,FOOTBALL,Schotse Premiership,M,Motherwell,Celtic
90235,2488757068,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021103898,3623883650,1,1,5.30,NaN,Motherwell,1,2025-02-01T20:57:17Z,17/4,430.0,OPEN,DISABLED,NaN,NaN,Motherwell vs Celtic,FOOTBALL,Schotse Premiership,M,Motherwell,Celtic


In [61]:
# Get unique records from 'Event Name' and 'matched_event'
matched_events = filtered_toto_winnaar[['Event Name', 'matched_event', 'fuzzy_score']].drop_duplicates()

# Display the result
matched_events[~matched_events['matched_event'].isna()].sort_values(by='fuzzy_score', ascending=False).to_csv('test_match.csv')

In [62]:
# Define a transformation function for standardization
def standardize_draw_no_bet(value):
    if '1e Helft' in value:
        return 'Draw No Bet - 1e Helft'
    elif '2e Helft' in value:
        return 'Draw No Bet - 2e Helft'
    elif 'Draw No Bet' in value:
        return 'Draw No Bet'
    else:
        return value  # Keep original if no match

# Apply the transformation to the relevant columns
filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)

# Merge the DataFrames using the standardized column
merged_winnaar = pd.merge(
    filtered_toto_winnaar,
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label'], 
    right_on=['event_name', 'standardized_label'], 
    how='inner'  # Perform an inner join
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/3072039294.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/3072039294.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)


In [63]:
# # Filter rows in toto_filtered_football where Market Name == 'Winnaar'
# toto_filtered_football = toto_filtered_football[toto_filtered_football['Market Name'].str.contains('Draw No Bet')]

# Perform a left join on the matched_event_name (from toto) and event_name (from kambi)
merged_df_winnaar = filtered_toto_winnaar.merge(
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label', 'sex'], 
    right_on=['event_name', 'standardized_label', 'sex'], 
    how='left'
)

merged_df_winnaar = merged_df_winnaar[(merged_df_winnaar['matched_event'].notnull())&(merged_df_winnaar['Outcome SubType'] != merged_df_winnaar['outcome_label'])]

## Over/Under

In [78]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_raw_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_basketbal based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_basketbal based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

In [79]:
kambi_filtered_football_overunder = kambi_filtered_football[kambi_filtered_football['bet_offer_type_name'].str.contains('Over')]
toto_filtered_football_overunder = toto_filtered_football[(toto_filtered_football['Outcome Name'].str.contains('Over')) | (toto_filtered_football['Outcome Name'].str.contains('Under'))]

In [80]:
toto_filtered_football['Market Name'].unique()

array(['Draw No Bet', 'Dubbele Kans', 'Resultaat',
       'Handicap Resultaat 1.0', 'Handicap Resultaat -3.0',
       'Handicap Resultaat -2.0', 'Aantal Goals - Over/Under 0.5',
       'Handicap Resultaat -1.0', 'Correct score Combinatie',
       'Beide Teams Scoren', 'Handicap Resultaat 2.0',
       'Aantal Goals - Over/Under 1.5', 'Aantal Goals - Over/Under 4.5',
       'Aantal Goals - Over/Under 3.5', 'Aantal Goals - Over/Under 2.5',
       'Half-Time / Full-Time', 'Correct Score',
       'Aantal Goals - Over/Under 5.5', 'TOTO Special',
       'Speler Scoort Op Elk Moment', '2e Helft - Doelpuntenmaker',
       '1e Helft - Doelpuntenmaker', 'Eerste doelpuntenmaker',
       'Resultaat - Vroege uitbetaling', 'Handicap Resultaat -4.0',
       'Speler scoort van buiten de 16 meter', 'Speler scoort kopgoal',
       'Price Boosts', 'Team schoten op doel',
       'Wedstrijd schoten op doel', 'Speler schoten op doel specials',
       'Player Specials', 'Handicap Resultaat 3.0', 'Match Specia

In [81]:
[i for i in kambi_filtered_football['criterion_label'].unique() if 'Schoten' in i]

['Totaal Aantal Schoten Door Toulouse (Afgehandeld Volgens Opta-Gegevens)',
 'Totaal Aantal Schoten op Doel (Afgehandeld Volgens Opta-Gegevens)',
 'Toulouse Raakt het Houtwerk in de Wedstrijd (Afgewikkeld volgens Opta-gegevens. Met uitzondering van situaties waarbij een doelpunt direct volgt na het raken van de paal of lat. Schoten tegen het houtwerk worden alleen toegeschreven aan het aanvallende team)',
 'Totaal Aantal Schoten (Afgehandeld Volgens Opta-Gegevens)',
 'Totaal Aantal Schoten op Doel door Toulouse (Afgehandeld Volgens Opta-Gegevens)',
 'Schoten van Speler (Afgehandeld Volgens Opta-Gegevens)',
 'Paal of Lat Worden Geraakt in de Wedstrijd (Afgewikkeld volgens Opta-gegevens. Met uitzondering van situaties waarbij een doelpunt direct volgt na het raken van de paal of lat. Schoten tegen het houtwerk worden alleen toegeschreven aan het aanvallende team)',
 'Schoten van Speler op Doel (Afgehandeld Volgens Opta-Gegevens)',
 'Totaal Aantal Schoten op Doel door Nice (Afgehandeld Vo

In [82]:
import unicodedata

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_english_label'] = kambi_filtered_football_overunder['criterion_english_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Event Name'] = toto_filtered_football_overunder['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Market Name'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

kambi_events = kambi_filtered_football_overunder['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score = result
#     return match


# # Apply matching function to toto_filtered_football
# toto_filtered_football_overunder['matched_event'] = toto_filtered_football_overunder['Event Name'].apply(find_best_match)
# Perform a left join to bring in 'matched_event' and 'fuzzy_score' from matched_events
toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
    matched_events,
    on='Event Name',  # Join on the 'Event Name' column
    how='left'  # Ensure all rows in toto_filtered_football_overunder are retained
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/2219985270.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/2219985270.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_r

In [83]:
# Get unique records from 'Event Name' and 'matched_event'
unique_records = toto_filtered_football_overunder[['Event Name', 'matched_event']].drop_duplicates()

# Display the result
unique_records[~unique_records['matched_event'].isna()]

,Event Name,matched_event
0,1. FC Kaiserslautern vs SC Preussen Munster,1. FC Kaiserslautern vs Preußen Munster
22,Hamburger SV vs Hannover 96,Hamburger SV vs Hannover 96
44,SC Paderborn vs SpVgg Greuther Furth,SC Paderborn 07 vs Greuther Furth
66,Barcelona vs Deportivo Alaves,FC Barcelona vs Alaves
88,Goztepe SK vs Alanyaspor,Goztepe A.S. vs Alanyaspor
...,...,...
7685,Sef Torres Calcio vs USD Sestri Levante 1919,Torres vs Sestri Levante
7721,USD Audace Cerignola vs Calcio Catania,Audace Cerignola vs Catania
7733,US Citta Di Pontedera vs ASD Pineto Calcio,Pontedera vs ASD Pineto Calcio
7745,US Pianese vs AS Lucchese Libertas 1905,Pianese vs Lucchese


In [84]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')  

In [ ]:
# Create 'OverUnderType'
kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: 'Goals' if ('Doelpunten' in x and 'Resultaat' not in x) else 'Speler schoten op doel' if 'Schoten van speler' in x 
    else 'Team schoten op doel' if 'Totaal Aantal Schoten door' in x else 'Wedstrijd schoten op doel' if 'Totaal Aantal Schoten op Doel' in x 
        else 'other'
)
)

# Create 'OverUnderTime'
def determine_over_under_time(label):
    if '1e helft' in label.lower():
        return '1e Helft'
    elif '2e helft' in label.lower():
        return '2e Helft'
    elif label[-2:].isdigit():  # Check if the last two characters in the label are digits
        return label.split(' - ')[-1]
    else:
        return 'Full Time'

kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

# Create 'Team1' and 'Team2'
kambi_filtered_football_overunder[['Team1', 'Team2']] = kambi_filtered_football_overunder['event_name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
    lambda row: '1' if row['Team1'] in row['criterion_english_label'] else (
        '2' if row['Team2'] in row['criterion_english_label'] else 'Total team 1 and team 2'
    ),
    axis=1
)

# # Create 'line'
# kambi_filtered_football_overunder['line'] = kambi_filtered_football_overunder['line'] / 1000

# # Creat odds
# kambi_filtered_football_overunder['odds'] = kambi_filtered_football_overunder['odds'] / 1000

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/3624939390.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_81757/3624939390.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)
/va

In [ ]:
# Create 'OverUnderType'
toto_filtered_football_overunder['OverUnderType'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: 'Goals' if ('Goals' in x and 'Resultaat' not in x) else 'Speler schoten op doel' if 'Speler schoten op doel' in x 
        else 'Team schoten op doel' if 'Team schoten op doel' in x else 'Wedstrijd schoten op doel' if 'Wedstrijd schoten op doel' in x 
        else 'other'
)

# Create 'OverUnderTime'
toto_filtered_football_overunder['OverUnderTime'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: '1e Helft' if '1e helft' in x.lower() else ('2e Helft' if '2e helft' in x.lower() else 'Full Time')
)

# Create 'Team1' and 'Team2'
toto_filtered_football_overunder[['Team1', 'Team2']] = toto_filtered_football_overunder['Event Name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
    lambda row: '1' if row['Team1'] in row['Market Name'] else (
        '2' if row['Team2'] in row['Market Name'] else 'Total team 1 and team 2'
    ),
    axis=1
)

# Create 'line'
toto_filtered_football_overunder['line'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: float(x.split('Over/Under ')[-1]) if 'Over/Under ' in x else None
)

In [93]:
# Perform the merge
merged_football_overunder = pd.merge(
    toto_filtered_football_overunder,
    kambi_filtered_football_overunder,
    left_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'matched_event', 'sex'],
    right_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'event_name', 'sex'],
    how='inner'  # Use 'inner' join to get only matching rows; adjust as needed
)
# Keep records with opposite outcomes
merged_football_overunder = merged_football_overunder[merged_football_overunder['outcome_english_label'] != merged_football_overunder['Outcome Name']]

# Display the merged DataFrame
merged_football_overunder.head()

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,match_name,home_team,away_team,sex,matched_event,fuzzy_score,OverUnderType,OverUnderTime,Team1_x,Team2_x,OverUnderType2,line,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y
1,1. FC Kaiserslautern vs SC Preussen Munster,Aantal Goals Over/Under 0.5,Over,1.03,3,100,HL,1,Voetbal,Duitsland 2.Bundesliga,1. FC Kaiserslautern vs SC Preussen Munster,1. FC Kaiserslautern,SC Preussen Munster,M,1. FC Kaiserslautern vs Preußen Munster,92.5,Goals,Full Time,1. FC Kaiserslautern,SC Preussen Munster,Total team 1 and team 2,0.5,2488808645,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021151143,3624303079,Minder dan,Under,15.00,NaN,OT_UNDER,2025-02-02T10:31:06Z,14/1,1400.0,OPEN,ENABLED,NaN,NaN,1. FC Kaiserslautern vs Preußen Munster,FOOTBALL,2. Bundesliga,1. FC Kaiserslautern,Preußen Munster
2,1. FC Kaiserslautern vs SC Preussen Munster,Aantal Goals Over/Under 0.5,Under,9.75,35,4,HL,L,Voetbal,Duitsland 2.Bundesliga,1. FC Kaiserslautern vs SC Preussen Munster,1. FC Kaiserslautern,SC Preussen Munster,M,1. FC Kaiserslautern vs Preußen Munster,92.5,Goals,Full Time,1. FC Kaiserslautern,SC Preussen Munster,Total team 1 and team 2,0.5,2488808645,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021151143,3624303074,Meer dan,Over,NaN,NaN,OT_OVER,2025-02-02T10:31:06Z,NaN,NaN,SUSPENDED,ENABLED,NaN,NaN,1. FC Kaiserslautern vs Preußen Munster,FOOTBALL,2. Bundesliga,1. FC Kaiserslautern,Preußen Munster
4,Hamburger SV vs Hannover 96,Aantal Goals Over/Under 4.5,Under,1.17,17,100,HL,L,Voetbal,Duitsland 2.Bundesliga,Hamburger SV vs Hannover 96,Hamburger SV,Hannover 96,M,Hamburger SV vs Hannover 96,100.0,Goals,Full Time,Hamburger SV,Hannover 96,Total team 1 and team 2,4.5,2488866626,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021151158,3624338542,Meer dan,Over,5.30,NaN,OT_OVER,2025-02-02T11:31:43Z,17/4,430.0,OPEN,ENABLED,NaN,NaN,Hamburger SV vs Hannover 96,FOOTBALL,2. Bundesliga,Hamburger SV,Hannover 96
7,Hamburger SV vs Hannover 96,Aantal Goals Over/Under 4.5,Over,4.50,7,2,HL,1,Voetbal,Duitsland 2.Bundesliga,Hamburger SV vs Hannover 96,Hamburger SV,Hannover 96,M,Hamburger SV vs Hannover 96,100.0,Goals,Full Time,Hamburger SV,Hannover 96,Total team 1 and team 2,4.5,2488866626,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021151158,3624338544,Minder dan,Under,1.13,NaN,OT_UNDER,2025-02-02T11:31:43Z,1/8,-770.0,OPEN,ENABLED,NaN,NaN,Hamburger SV vs Hannover 96,FOOTBALL,2. Bundesliga,Hamburger SV,Hannover 96
9,SC Paderborn vs SpVgg Greuther Furth,Aantal Goals Over/Under 0.5,Over,1.02,1,50,HL,1,Voetbal,Duitsland 2.Bundesliga,SC Paderborn vs SpVgg Greuther Fürth,SC Paderborn,SpVgg Greuther Fürth,M,SC Paderborn 07 vs Greuther Furth,100.0,Goals,Full Time,SC Paderborn,SpVgg Greuther Furth,Total team 1 and team 2,0.5,2488820189,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021151155,3624327280,Minder dan,Under,17.00,NaN,OT_UNDER,2025-01-31T08:58:56Z,16/1,1600.0,OPEN,ENABLED,NaN,NaN,SC Paderborn 07 vs Greuther Furth,FOOTBALL,2. Bundesliga,SC Paderborn 07,Greuther Furth


## Match Winnaar & OverUnder and do Arb Analysis

In [94]:
# Perform the stacked union
total_football = pd.concat([merged_football_overunder, merged_df_winnaar], ignore_index=True, sort=True)
total_football

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,away_team,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id,event_name,fuzzy_score,group_name,home_score,home_team,lifetime,line,match_name,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,status,type
0,1. FC Kaiserslautern vs SC Preussen Munster,Aantal Goals Over/Under 0.5,1.03,Over,1,HL,Full Time,Goals,Total team 1 and team 2,100,3,1. FC Kaiserslautern,1. FC Kaiserslautern,SC Preussen Munster,Preußen Munster,NaN,SC Preussen Munster,2.488809e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-02T10:31:06Z,Duitsland 2.Bundesliga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021151e+09,1. FC Kaiserslautern vs Preußen Munster,92.5,2. Bundesliga,NaN,1. FC Kaiserslautern,FULL_TIME,0.5,1. FC Kaiserslautern vs SC Preussen Munster,1. FC Kaiserslautern vs Preußen Munster,GOALS,15.00,1400.0,14/1,Under,3.624303e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_UNDER
1,1. FC Kaiserslautern vs SC Preussen Munster,Aantal Goals Over/Under 0.5,9.75,Under,L,HL,Full Time,Goals,Total team 1 and team 2,4,35,1. FC Kaiserslautern,1. FC Kaiserslautern,SC Preussen Munster,Preußen Munster,NaN,SC Preussen Munster,2.488809e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-02T10:31:06Z,Duitsland 2.Bundesliga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021151e+09,1. FC Kaiserslautern vs Preußen Munster,92.5,2. Bundesliga,NaN,1. FC Kaiserslautern,FULL_TIME,0.5,1. FC Kaiserslautern vs SC Preussen Munster,1. FC Kaiserslautern vs Preußen Munster,GOALS,NaN,NaN,NaN,Over,3.624303e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,SUSPENDED,OT_OVER
2,Hamburger SV vs Hannover 96,Aantal Goals Over/Under 4.5,1.17,Under,L,HL,Full Time,Goals,Total team 1 and team 2,100,17,Hamburger SV,Hamburger SV,Hannover 96,Hannover 96,NaN,Hannover 96,2.488867e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-02T11:31:43Z,Duitsland 2.Bundesliga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021151e+09,Hamburger SV vs Hannover 96,100.0,2. Bundesliga,NaN,Hamburger SV,FULL_TIME,4.5,Hamburger SV vs Hannover 96,Hamburger SV vs Hannover 96,GOALS,5.30,430.0,17/4,Over,3.624339e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_OVER
3,Hamburger SV vs Hannover 96,Aantal Goals Over/Under 4.5,4.50,Over,1,HL,Full Time,Goals,Total team 1 and team 2,2,7,Hamburger SV,Hamburger SV,Hannover 96,Hannover 96,NaN,Hannover 96,2.488867e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-02T11:31:43Z,Duitsland 2.Bundesliga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021151e+09,Hamburger SV vs Hannover 96,100.0,2. Bundesliga,NaN,Hamburger SV,FULL_TIME,4.5,Hamburger SV vs Hannover 96,Hamburger SV vs Hannover 96,GOALS,1.13,-770.0,1/8,Under,3.624339e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_UNDER
4,SC Paderborn vs SpVgg Greuther Furth,Aantal Goals Over/Under 0.5,1.02,Over,1,HL,Full Time,Goals,Total team 1 and team 2,50,1,SC Paderborn,SC Paderborn 07,SpVgg Greuther Furth,Greuther Furth,NaN,SpVgg Greuther Fürth,2.488820e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-31T08:58:56Z,Duitsland 2.Bundesliga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021151e+09,SC Paderborn 07 vs Greuther Furth,100.0,2. Bundesliga,NaN,SC Paderborn,FULL_TIME,0.5,SC Paderborn vs SpVgg Greuther Fürth,SC Paderborn 07 vs Greuther Furth,GOALS,17.00,1600.0,16/1,Under,3.624327e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_UNDER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,US Citta Di Pontedera vs ASD Pineto Calcio,Draw No Bet,1.

In [95]:
# Calculate Arbitrage Percentage
total_football['Arbitrage Percentage'] = (1 / total_football['Odds (Decimal)'] + 1 / total_football['odds']) * 100

# Identify Arbitrage Opportunities
total_football['Is Arbitrage'] = total_football['Arbitrage Percentage'] < 100

# Calculate Optimal Stakes if Arbitrage
total_stake = 1000
total_football['Stake A'] = total_football.apply(
    lambda row: (1 / row['Odds (Decimal)'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)
total_football['Stake B'] = total_football.apply(
    lambda row: (1 / row['odds'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)

total_football[['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]    

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
0,1. FC Kaiserslautern vs SC Preussen Munster,Aantal Goals Over/Under 0.5,Over,Minder dan,1.03,15.00,103.754045,False,0.0,0.0
1,1. FC Kaiserslautern vs SC Preussen Munster,Aantal Goals Over/Under 0.5,Under,Meer dan,9.75,NaN,NaN,False,0.0,0.0
2,Hamburger SV vs Hannover 96,Aantal Goals Over/Under 4.5,Under,Meer dan,1.17,5.30,104.338010,False,0.0,0.0
3,Hamburger SV vs Hannover 96,Aantal Goals Over/Under 4.5,Over,Minder dan,4.50,1.13,110.717797,False,0.0,0.0
4,SC Paderborn vs SpVgg Greuther Furth,Aantal Goals Over/Under 0.5,Over,Minder dan,1.02,17.00,103.921569,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
873,US Citta Di Pontedera vs ASD Pineto Calcio,Draw No Bet,ASD Pineto Calcio,1,1.93,1.74,109.284736,False,0.0,0.0
874,US Pianese vs AS Lucchese Libertas 1905,Draw No Bet,AS Lucchese Libertas 1905,1,2.45,1.52,106.605800,False,0.0,0.0
875,US Pianese vs AS Lucchese Libertas 1905,Draw No Bet,US Pianese,2,1.40,2.35,113.981763,False,0.0,0.0
876,Pescara vs Virtus Entella,Draw No Bet,Virtus Entella,1,1.91,1.70,111.179550,False,0.0,0.0


In [96]:
total_football[total_football['Is Arbitrage'] == True][['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]  

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
657,KRC Genk vs Cercle Brugge,Draw No Bet,Cercle Brugge,1,3.6,2.1,75.396825,True,368.421053,631.578947


In [97]:
total_football[(total_football['Is Arbitrage'] == True)][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'outcome_label', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B', 'group_name', 'sex']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,outcome_label,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B,group_name,sex
657,KRC Genk vs Cercle Brugge,KRC Genk vs Club Brugge,Draw No Bet,Draw No Bet,Cercle Brugge,1,NaN,3.6,2.1,13,5,DN,2,Voetbal,België Jupiler Pro League,NaN,NaN,KRC Genk,Cercle Brugge,KRC Genk,Club Brugge,75.396825,True,368.421053,631.578947,Beker van Belgie,M
